In [1]:
import torch.optim as optim
import torch, torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.autograd import Variable
from models import CNN
from pathlib import Path
from MLDataTools.image_normalization import RandomDihedral
from skimage.external import tifffile as tiff
import numpy as np

torch.set_default_tensor_type(torch.DoubleTensor)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # important for cloud compatability

## Initialize the train loader and the test loader

In [2]:
DATA_ROOT = '/Users/cerber/HDev Dropbox/Projects/YNet_ready_data/yeast_v4'
data_path = Path(DATA_ROOT)

In [3]:
def tiff_read(path:str):
    image = tiff.imread(path).astype(np.double)
    return image

class GetInfo:
    def __init__(self, label=None):
        self.label = label
    def __call__(self, sample):
        try:
            print(sample.shape)
        except: pass
        finally:
            if self.label: print(self.label)
            print(type(sample))
            return sample


In [4]:
ds_transforms = transforms.Compose([
    RandomDihedral()
])
trainset = torchvision.datasets.ImageFolder(DATA_ROOT+'/train',transform=ds_transforms, loader=tiff_read)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=40, shuffle=True, num_workers=0)

testset = torchvision.datasets.ImageFolder(DATA_ROOT+'/test', loader=tiff_read)
testloader = torch.utils.data.DataLoader(testset, batch_size=40, shuffle=False, num_workers=0)

In [5]:
# class_name: (mean, stdev)
stats = {
    'WT': ([48.37304926, 95.64728183],
  [521.95544101, 310.77445807]),
    'mfb1KO': ([ 47.58621839, 102.40188124],
  [520.43241635, 311.95406937]),
    'mfb1KO_mmr1KO': ([ 47.79873863, 100.28439551],
  [517.82433373, 310.53787264]),
    'mmr1KO': ([ 49.22677943, 110.97112597],
  [522.00261751, 315.86275802])
}

# invert class_to_id
idx_to_class = {v:k for k,v in trainset.class_to_idx.items()}
norm_transforms = {}
for key, value in stats.items():
    label = trainset.class_to_idx[key]
    norm_transforms[label] = transforms.Normalize(value[0],value[1])

In [6]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(2, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(32 * 47 * 47, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 4)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 32 * 47 * 47)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [8]:

def train(epoch):
    net.train() # affects only modules like Dropout
    trainiter = iter(trainloader)
    for batch_idx, (data, targets) in enumerate(trainiter, 0):
        # get the inputs
        for i, target in enumerate(targets, 0): # normalize the inputs according to class
            t = norm_transforms[target.item()]
            data[i] = t(data[i])

        data, targets = data.to(device), targets.to(device)

        optimizer.zero_grad() # dont forget to do that
        output = net(data)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 5 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(trainloader.dataset)} "
                  f"({100. * batch_idx / len(trainloader)}%)]\tLoss: {loss.item()}")
#
# A simple test procedure to measure STN the performances on MNIST.
#


def test():
    with torch.no_grad():
        net.eval()
        test_loss = 0
        correct = 0
        for data, targets in iter(testloader):
            data, targets = data.to(device), targets.to(device)
            for i, target in enumerate(targets, 0): # normalize the inputs according to class
                t = norm_transforms[target.item()]
                data[i] = t(data[i])
            output = net(data)
            # sum up batch loss
            test_loss += criterion(output, targets).item()
            # get the index of the max log-probability
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(targets.view_as(pred)).sum().item()

        test_loss /= len(testloader.dataset)
        print(f"\nTest set: Average loss: {test_loss:{5}}, Accuracy: {correct}/{len(testloader.dataset)}"
              f" ({100. * correct / len(testloader.dataset):{5}}%)\n")

In [ ]:
EPOCHS = 20
for i in range(EPOCHS):
    train(i)
    test()
    
print("Finisehd training.")

Train Epoch: 0 [0/609 (0.0%)]	Loss: 1.407597885071891
Train Epoch: 0 [200/609 (31.25%)]	Loss: 1.3810487205808046
Train Epoch: 0 [400/609 (62.5%)]	Loss: 1.2947981341883796
Train Epoch: 0 [135/609 (93.75%)]	Loss: 1.3485243313180173

Test set: Average loss: 0.039941242079823705, Accuracy: 24/105 (22.857142857142858%)

Train Epoch: 1 [0/609 (0.0%)]	Loss: 1.328797571958806
Train Epoch: 1 [200/609 (31.25%)]	Loss: 1.4059843855310992
Train Epoch: 1 [400/609 (62.5%)]	Loss: 1.3980572087286187
